In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [3]:
! pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-multimodal 0.8.3 requires pytorch-lightning<1.10.0,>=1.9.0, but you have pytorch-lightning 2.0.9 which is incompatible.
autogluon-multimodal 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 0.8.3 requires torchmetrics<0.12.0,>=0.11.0, but you have torchmetrics 1.0.3 which is incompatible.
autogluon-multimodal 0.8.3 requires torchvision<0.15.0, but you have torchvision 0.15.2a0+ab7b3e6 which is incompatible.
autogluon-timeseries 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-timeseries 0.8.3 requires pytorch-lightning<1.10.0,>=1.7.4, but you have pytorch-lightning 2.0.9 which is incompatib

In [4]:
! pip install transformers==4.27.2 datasets==2.11.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-multimodal 0.8.3 requires pytorch-lightning<1.10.0,>=1.9.0, but you have pytorch-lightning 2.0.9 which is incompatible.
autogluon-multimodal 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 0.8.3 requires torchmetrics<0.12.0,>=0.11.0, but you have torchmetrics 1.0.3 which is incompatible.
autogluon-multimodal 0.8.3 requires torchvision<0.15.0, but you have torchvision 0.15.2a0+ab7b3e6 which is incompatible.
autogluon-multimodal 0.8.3 requires transformers[sentencepiece]<4.41.0,>=4.36.0, but you have transformers 4.27.2 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.6 which is incompatible.
pathos 0.3.2

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Summarize Dialogue without Prompt Engineering

### Here we will be generating a summary of a dialogue with the pre-trained LLM FLAN-T5 from Hugging Face
### Let's upload some simple dialogues from DialogSum Hugging Face dataset. This dataset contains 10k+ dialogues with the corresponding manually labelled summaries and topics

In [2]:
huggingface_dataset_name='knkarthick/dialogsum'
dataset= load_dataset(huggingface_dataset_name)

Found cached dataset csv (/home/sagemaker-user/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
## print a couple of dialogues with their baseline summaries
example_indices =[40,200]
dash_line = '-'.join('' for x in range(100))
for i,index in enumerate(example_indices):
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

In [4]:
# now we will improve the summary by our model

### Load the Flan-T5 model, create an instance of the AutoModelForSeq2SeqLM class with .from_pretrained() method

In [5]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### To perform encoding and decoding, you need the text in tokenized form. Tokenization is the process of spliting text into smaller units that can be processed by LLM models.

#### Download the tokenizer for flan-t5 model using AutoTokenizer.from_pretrained() method. Parameter use_fast switches on fast tokenizer.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [7]:
## These are all from the hugging face transformer library

#### Test the tokenizer encoding and decoding a sentence

In [8]:
sentence = 'What time is it, Tom?'
sentence_encoded = tokenizer(sentence,return_tensors='pt')

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)
print("Encoded Sentence:")
print(sentence_encoded['input_ids'][0])
print('\n Decoded Sentence:')
print(sentence_decoded)

2024-09-16 03:44:29.778213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Encoded Sentence:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

 Decoded Sentence:
What time is it, Tom?


#### Now, it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. Prompt engineering is an act of human changing the prompt(input) to improve the response for a given task 

In [9]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    inputs = tokenizer(dialogue,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
 Person1: It's ten to nine.

------------------------------

In [10]:
# It is not doing a very good job

## Summarize Dialogue with an Instruction Prompt

In [11]:
## Prompt engineering is an important concept in using foundation models for text generation.

### Zero Shot Inference with an Instruction Prompt

#### In order to instruct the model to perform a task - summarise a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called zero shot inference. 

In [12]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt= f"""

    Summarize the following conversation.
    {dialogue}
    Summary:
    """
    #input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:


    Summarize the following conversation.
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
 The train

In [13]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt= f"""

    Dialogue:
    {dialogue}

    What was going on?
    """
    #input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:


    Dialogue:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What was going on?
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
 Tom is late for the train.

## Summarize dialogue with One Shot and Few Shot Inference

#### One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task before your actual prompt that you want completed. This is called "in-context" learning and puts your model into a state that understands your specific task. 

In [14]:
## One Shot Inference

In [19]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence
        prompt += f"""
        Dialogue:
        {dialogue}

        What was going on?
        {summary}
        """

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""

    Dialogue:
    {dialogue}
    What was going on?
    """
   
    return prompt

In [20]:
# construct the prompt to perform one shot inference

In [21]:
example_indices_full =[40]
example_index_to_summarize = 200
one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        

    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a fas

In [22]:
## Now pass the prompt to perform the one shot inference

In [23]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors="pt")
output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
 #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.



In [24]:
## Prompt engineering, zero shot, one shot, few shot inferences are the first step always if we have a hub of models

## Few Shot Inferences

In [25]:
# we are helping the model
example_indices_full = [40,80,120]
example_index_to_summarize = 200
few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable p

In [27]:
generation_config = GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.0)

In [28]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors="pt")
output = tokenizer.decode(model.generate(inputs['input_ids'],generation_config,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
 #Person1 suggests the following software enhancements to their software: adding a painting program, adding a computer software software, speeding up the computer, adding a CD, adding a CD-ROM drive.



#### In this case few shot did not provide much of an improvement over one shot inference. And, anything above 5 or 6 shot will typically not help much, either. Also, you need to ensure that you do not exceed the model's input-context length which, in our case, if 512 tokens. Anything above the context length will be ignored. 

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [3]:
! pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-multimodal 0.8.3 requires pytorch-lightning<1.10.0,>=1.9.0, but you have pytorch-lightning 2.0.9 which is incompatible.
autogluon-multimodal 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 0.8.3 requires torchmetrics<0.12.0,>=0.11.0, but you have torchmetrics 1.0.3 which is incompatible.
autogluon-multimodal 0.8.3 requires torchvision<0.15.0, but you have torchvision 0.15.2a0+ab7b3e6 which is incompatible.
autogluon-timeseries 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-timeseries 0.8.3 requires pytorch-lightning<1.10.0,>=1.7.4, but you have pytorch-lightning 2.0.9 which is incompatib

In [4]:
! pip install transformers==4.27.2 datasets==2.11.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-multimodal 0.8.3 requires pytorch-lightning<1.10.0,>=1.9.0, but you have pytorch-lightning 2.0.9 which is incompatible.
autogluon-multimodal 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 0.8.3 requires torchmetrics<0.12.0,>=0.11.0, but you have torchmetrics 1.0.3 which is incompatible.
autogluon-multimodal 0.8.3 requires torchvision<0.15.0, but you have torchvision 0.15.2a0+ab7b3e6 which is incompatible.
autogluon-multimodal 0.8.3 requires transformers[sentencepiece]<4.41.0,>=4.36.0, but you have transformers 4.27.2 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.6 which is incompatible.
pathos 0.3.2

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Summarize Dialogue without Prompt Engineering

### Here we will be generating a summary of a dialogue with the pre-trained LLM FLAN-T5 from Hugging Face
### Let's upload some simple dialogues from DialogSum Hugging Face dataset. This dataset contains 10k+ dialogues with the corresponding manually labelled summaries and topics

In [2]:
huggingface_dataset_name='knkarthick/dialogsum'
dataset= load_dataset(huggingface_dataset_name)

Found cached dataset csv (/home/sagemaker-user/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
## print a couple of dialogues with their baseline summaries
example_indices =[40,200]
dash_line = '-'.join('' for x in range(100))
for i,index in enumerate(example_indices):
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

In [4]:
# now we will improve the summary by our model

### Load the Flan-T5 model, create an instance of the AutoModelForSeq2SeqLM class with .from_pretrained() method

In [5]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### To perform encoding and decoding, you need the text in tokenized form. Tokenization is the process of spliting text into smaller units that can be processed by LLM models.

#### Download the tokenizer for flan-t5 model using AutoTokenizer.from_pretrained() method. Parameter use_fast switches on fast tokenizer.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [7]:
## These are all from the hugging face transformer library

#### Test the tokenizer encoding and decoding a sentence

In [8]:
sentence = 'What time is it, Tom?'
sentence_encoded = tokenizer(sentence,return_tensors='pt')

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)
print("Encoded Sentence:")
print(sentence_encoded['input_ids'][0])
print('\n Decoded Sentence:')
print(sentence_decoded)

2024-09-16 03:44:29.778213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Encoded Sentence:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

 Decoded Sentence:
What time is it, Tom?


#### Now, it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. Prompt engineering is an act of human changing the prompt(input) to improve the response for a given task 

In [9]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    inputs = tokenizer(dialogue,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
 Person1: It's ten to nine.

------------------------------

In [10]:
# It is not doing a very good job

## Summarize Dialogue with an Instruction Prompt

In [11]:
## Prompt engineering is an important concept in using foundation models for text generation.

### Zero Shot Inference with an Instruction Prompt

#### In order to instruct the model to perform a task - summarise a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called zero shot inference. 

In [12]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt= f"""

    Summarize the following conversation.
    {dialogue}
    Summary:
    """
    #input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:


    Summarize the following conversation.
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
 The train

In [13]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt= f"""

    Dialogue:
    {dialogue}

    What was going on?
    """
    #input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt,return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example ',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:


    Dialogue:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What was going on?
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
 Tom is late for the train.

## Summarize dialogue with One Shot and Few Shot Inference

#### One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task before your actual prompt that you want completed. This is called "in-context" learning and puts your model into a state that understands your specific task. 

In [14]:
## One Shot Inference

In [19]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence
        prompt += f"""
        Dialogue:
        {dialogue}

        What was going on?
        {summary}
        """

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""

    Dialogue:
    {dialogue}
    What was going on?
    """
   
    return prompt

In [20]:
# construct the prompt to perform one shot inference

In [21]:
example_indices_full =[40]
example_index_to_summarize = 200
one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        

    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a fas

In [22]:
## Now pass the prompt to perform the one shot inference

In [23]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors="pt")
output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens=50,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
 #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.



In [24]:
## Prompt engineering, zero shot, one shot, few shot inferences are the first step always if we have a hub of models

## Few Shot Inferences

In [25]:
# we are helping the model
example_indices_full = [40,80,120]
example_index_to_summarize = 200
few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable p

In [27]:
generation_config = GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.0)

In [28]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors="pt")
output = tokenizer.decode(model.generate(inputs['input_ids'],generation_config,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n {output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
 #Person1 suggests the following software enhancements to their software: adding a painting program, adding a computer software software, speeding up the computer, adding a CD, adding a CD-ROM drive.



#### In this case few shot did not provide much of an improvement over one shot inference. And, anything above 5 or 6 shot will typically not help much, either. Also, you need to ensure that you do not exceed the model's input-context length which, in our case, if 512 tokens. Anything above the context length will be ignored. 